<a href="https://colab.research.google.com/github/PabloViniciusSS/forward_backward_rede_neural/blob/main/criacao_rede_neural.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Projeto para criação de uma rede neural, criando as funções manuais de todos os passos para a criação, sem a utilização de um framework.
Utilizando a vetorização para melhorar o desempenho.


In [1]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
import copy

np.random.seed(1)

Irei criar as funções para uma rede neural, elas poderão ser utilizadas em outras redes.

* Passa a passo:

  - Inicialmente faremos as funções de ativações, para Forward, e gerar ua suposição "y_hat"
  - E depois para Backward para atualizar os parametros do sistema

In [2]:
def sigmoid(Z):
  A = 1/(1+np.exp(-Z))
  cache = Z

  return A, cache

In [15]:
def relu(Z):
  A = np.maximun(0, Z)

  assert(A.shape == Z.shape)

  cache = Z

  return A, cache

In [16]:
def relu_backward(dA, cache):

  Z = cache
  dZ = np.Array(dA, copy=True)

  dZ[Z <= 0 ] = 0

  assert (dZ.shape == Z.shape)

  return dZ

In [17]:
def sigmoid_backward(dA, cache):
  Z = cache

  s = 1/(1+np.exp(-Z))
  dZ = dA * s * (1-s)

  assert (dZ.shape == Z.shape)

  return dZ


Agora vamos criar as funções para a rede neural.

Para a "forward propagation":

* A Rede neural contará com duas camadas ocultas, mais a camada L

* As ativações utilizadas serão o sigmoid e o relu

* Irá computar a perda

Para a "backward propagation":

* Será feito todo o processo inverso.

* Calculará com base nas atiavações e nos caches.

* No fim será feito a atualização do W e b

Vamos iniciar criar os parametros, W e b, como seão duas camadas, teremos W1 e W2, e b1 e b2.

O parametro b pode ser zerado que não causa conflito na hora de prever, deixando de lado e facilitando na hora de atualizar.



In [5]:
def initializate_parameters(n_x, n_h, n_y):

  # n_x: dimensão da camada de entrada
  # n_h: dimensão da camada oculta
  # n_y: dimensão da camada de saida

  np.random.seed(2)
  W1 = np.random.randn(n_h, n_x) * 0.01
  b1 = np.zeros((n_h, 1))
  W2 = np.random.randn(n_y, n_h) * 0.01
  b2 = np.zeros((n_y, 1))


  parameters = {
                'W1': W1,
                'b1': b1,
                'W2': W2,
                'b2': b2
              }

  return parameters

O proximo é uma função automatica para gerar os parametros W e b, na quantidade que deseja, fazendo o sistema ser mais flexivel

In [6]:
def initialize_parameters_deep(layer_dims):

  np.random.seed(3)
  parameters = {}
  L = len(layer_dims)

  for l in range(1, L):
    parameters["W" + str(L)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01
    parameters["b" + str(L)] = np.zeros(layer_dims[l], 1)

    assert(parameters["W" + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
    assert(parameters["b" + str(l)].shape == (layer_dims[l],1))


  return parameters



Função para a criação da função linear

In [7]:
def linear_forward(A, W, b):

  Z = np.dot(W,A)+b

  cache = (A, W, b)

  return Z, cache

Agora criando a função das ativações, com um if para o sistema gerar a partir da que deseja, no caso "sigmoid" e "relu"

In [8]:
def linear_activation_forward(A_prev, W, b, activation):
  if activation == "sigmoid":
    Z, linear_cache = linear_forward(A_prev, W, b)
    A, activation_cache = sigmoid(Z)

  elif activation == "relu":
    Z, linear_cache = linear_forward(A_prev, W, b)

  cache = (linear_cache, activation_cache)
  return A, cache

Agora concluindo a fase de propagation forward, criando a predição

In [9]:
def L_model_forward(X, parameters):
  caches = []
  A = X
  L = len(parameters) // 2

  for l in range(1, L):
    A_prev = A

    A, cache = linear_activation_forward(A_prev,
                                         parameters['W'+str(l)],
                                         parameters['b'+str(l)],
                                         activation = 'relu')


    caches.append(cache)

    AL, cache = linear_activation_forward(A,
                                          parameters['W' + str(L)],
                                          parameters['b' + str(L)],
                                          activation='sigmoid')
    caches.append(cache)

    return AL, caches


E por fim fazendo o calculo do custo

In [10]:
def compute_cost(AL, Y):

  m = Y.shape[1]

  cost = (-1. / m) * np(np.multiply(Y, np.log(AL)) + np.multiply((1-Y), np.log(1-AL)))

  cost = np.squeeze(cost)

  return cost

Iniciando a fase de propagation Backward, iniciamos criando o dW, db e dA_prev

In [11]:
def linear_backward(dZ, cache):

  A_prev, W, b = cache
  m = A_prev.shape[1]

  dW = (1./m) * np.dot(dZ, cache[0].T)
  db = (1./m * np.sum(dZ,axis = 1, keepdims=True))
  dA_prev = np.dot(cache[1].T, dZ)

  return dA_prev, dW, db

Criando a função para calular as ativações

In [12]:
def linear_activation_backward(dA, cache, activation):

  linear_cache, activation_cache = cache

  if activation == "relu":

    dZ = relu_backward(dA, activation_cache)
    dA_prev, dW, db = linear_backward(dZ, linear_cache)

  elif activation == "sigmoid":

    dZ = sigmoid_backward(dA, activation_cache)
    dA_prev, dW, db = linear_backward(dZ, linear_cache)

    return dA_prev, dW, db

E por fim pegando os valor de dA e calculando a ativação relu e sigmoid

In [13]:
def L_model_backward(AL, Y, caches):
  grads = {}
  L = len(caches)
  m = AL.shapes[1]
  Y = Y.reshapes(AL.shape)

  dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))

  current_cache = caches[L-1]
  grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL, current_cache, activation = 'sigmoid')

  for l in reversed(range(L-1)):

    current_cache = caches[l]
    dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l + 1)], current_cache, activation="relu")
    grads["dA" + str(l)] = dA_prev_temp
    grads["dW" + str(l + 1)] = dW_temp
    grads["db" + str(l + 1)] = db_temp

    return grads

E por fim atualizando os parametros W e b

In [14]:
def update_parameters(params, grads, learning_rate):
  parameters = copy.deepcopy(params)
  L = len(parameters)

  for l in range(L):

    parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate * grads["dW" + str(l+1)]
    parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate * grads["db" + str(l+1)]

    return parameters

O projeto tem como finalidade aprender como função a rede neural, foi feito com base no curso de Deep Learning da DeepLearning.ia.

